# Hidden Markov models - Viterbi Algorithm

In [16]:
import math
import numpy as np
import pandas as pd
from collections import defaultdict
from utils import get_word_tag, preprocess  
from markov import get_counters, get_transition_mat, get_emission_mat

Training corpus structure
```word\t\tag\n```

In [2]:
with open('data/WSJ_02-21.pos', 'r') as f:
    training_corpus = f.readlines()

print(training_corpus[:5])    

['In\tIN\n', 'an\tDT\n', 'Oct.\tNNP\n', '19\tCD\n', 'review\tNN\n']


In [3]:
with open('data/hmm_vocab.txt', 'r') as f:
    raw_vocab = f.read().split('\n')

print('Some words from raw_vocab')
print(f'From beginning: {raw_vocab[:10]}')
print(f'From end: {raw_vocab[-10:]}')

Some words from raw_vocab
From beginning: ['!', '#', '$', '%', '&', "'", "''", "'40s", "'60s", "'70s"]
From end: ['zeros', 'zinc', 'zip', 'zombie', 'zone', 'zones', 'zoning', '{', '}', '']


```word -> index``` mapping

In [4]:
vocab = {word:i for i, word in enumerate(sorted(raw_vocab))}
list(vocab.items())[:10]

[('', 0),
 ('!', 1),
 ('#', 2),
 ('$', 3),
 ('%', 4),
 ('&', 5),
 ("'", 6),
 ("''", 7),
 ("'40s", 8),
 ("'60s", 9)]

test corpus structure
```word\t\tag\n```

In [5]:
with open("data/WSJ_24.pos", 'r') as f:
    test_corpus = f.readlines()
test_corpus[:5]

['The\tDT\n', 'economy\tNN\n', "'s\tPOS\n", 'temperature\tNN\n', 'will\tMD\n']

preprocess all words by:
1) if end of line, return --n--
2) if word valid but not in vocab, return most meaningful tag 
3) if word valid and in vocab, return word

In [6]:
orig, prep = preprocess(vocab, "data/test.words")     
orig[:20], prep[:20]

(['The',
  'economy',
  "'s",
  'temperature',
  'will',
  'be',
  'taken',
  'from',
  'several',
  'vantage',
  'points',
  'this',
  'week',
  ',',
  'with',
  'readings',
  'on',
  'trade',
  ',',
  'output'],
 ['The',
  'economy',
  "'s",
  'temperature',
  'will',
  'be',
  'taken',
  'from',
  'several',
  '--unk--',
  'points',
  'this',
  'week',
  ',',
  'with',
  'readings',
  'on',
  'trade',
  ',',
  'output'])

In [7]:
emission_counts, transition_counts, tag_counts = get_counters(training_corpus, vocab)

#### Transition Counts

Transition counts structure: {(prev_tag, tag), count}

This counter dictionary provides to tuple -> counter. 
Tuple first item is previous tag and the second one is the tag that followed by first one.

You can think the counter array as: 
* rows <-> all tags that represent previous tag 
* columns <-> all tags that comes after the previous tag

This dictionary helps us to create transition matrix.

In [8]:
print(f'Length of transition counts: {len(transition_counts)}')
list(transition_counts.items())[:10]

Length of transition counts: 1421


[(('--s--', 'IN'), 5050),
 (('IN', 'DT'), 32364),
 (('DT', 'NNP'), 9044),
 (('NNP', 'CD'), 1752),
 (('CD', 'NN'), 7377),
 (('NN', 'IN'), 32885),
 (('IN', '``'), 546),
 (('``', 'DT'), 1014),
 (('DT', 'NN'), 38873),
 (('NN', "''"), 686)]

#### Emission counts

Emission counts gives us the word counter if the tag is given. 
You can think the counter array as: 
* rows <-> tags
* columns <-> words

In [9]:
print(f'Length of emission counts: {len(emission_counts)}')
print(list(emission_counts.items())[:10])

Length of emission counts: 31140
[(('IN', 'In'), 1735), (('DT', 'an'), 3142), (('NNP', 'Oct.'), 317), (('CD', '19'), 100), (('NN', 'review'), 36), (('IN', 'of'), 22925), (('``', '``'), 6967), (('DT', 'The'), 6795), (('NN', 'Misanthrope'), 3), (("''", "''"), 6787)]


As you can see below, If the given tag is determiner -DT-, there are tons of words that might be. 

In [10]:
for tuple, count in list(emission_counts.items()):
    if tuple[0] == 'DT':
        print(tuple, count)

('DT', 'an') 3142
('DT', 'The') 6795
('DT', 'the') 41098
('DT', 'a') 19264
('DT', 'A') 817
('DT', 'some') 1274
('DT', 'this') 1897
('DT', 'any') 721
('DT', 'those') 505
('DT', 'Both') 99
('DT', 'Some') 314
('DT', 'no') 606
('DT', 'An') 141
('DT', 'Either') 3
('DT', 'This') 398
('DT', 'these') 417
('DT', 'another') 351
('DT', 'that') 1168
('DT', 'That') 412
('DT', 'each') 356
('DT', 'every') 171
('DT', 'all') 842
('DT', 'No') 82
('DT', 'both') 336
('DT', 'These') 139
('DT', 'Another') 72
('DT', 'Those') 66
('DT', 'Each') 57
('DT', 'Any') 18
('DT', 'All') 82
('DT', 'THE') 35
('DT', 'either') 50
('DT', 'many') 14
('DT', 'Every') 20
('DT', 'neither') 18
('DT', 'NO') 2
('DT', 'half') 31
('DT', 'Many') 2
('DT', 'Neither') 13
('DT', 'nary') 1
('DT', 'AN') 6
('DT', 'them') 1
('DT', 'la') 2
('DT', 'Half') 1
('DT', '--unk_upper--') 2
('DT', 'del') 1


Also it can be seen, There are ambigious words. If the given word Part of Speec Tag is `RB`, the number of times that `back` word is 304. However for `VB` is 20. 


In [11]:
for tuple, count in list(emission_counts.items()):
    if tuple[1] == 'back':
        print(tuple, count)

('RB', 'back') 304
('VB', 'back') 20
('RP', 'back') 84
('JJ', 'back') 25
('NN', 'back') 29
('VBP', 'back') 4


There is 46 Part of Speech Tag available in our corpus. 

In [12]:
states = sorted(tag_counts.keys())
print(f"Number of POS tags (number of 'states'): {len(states)}")
print("Let's see these states:")
print(states)

Number of POS tags (number of 'states'): 46
Let's see these states:
['#', '$', "''", '(', ')', ',', '--s--', '.', ':', 'CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB', '``']


In [13]:
alpha = 0.001
A = get_transition_mat(transition_counts, tag_counts, alpha)

print(f"A at row 0, col 0: {A[0,0]:.9f}")
print(f"A at row 3, col 1: {A[3,1]:.4f}")

A at row 0, col 0: 0.000007040
A at row 3, col 1: 0.1691


In [17]:
pd.DataFrame(A[30:35,30:35], index=states[30:35], columns = states[30:35] )

,RBS,RP,SYM,TO,UH
RBS,2.217069e-06,2.217069e-06,2.217069e-06,0.008870,2.217069e-06
RP,3.756509e-07,7.516775e-04,3.756509e-07,0.051089,3.756509e-07
SYM,1.722772e-05,1.722772e-05,1.722772e-05,0.000017,1.722772e-05
TO,4.477336e-05,4.472863e-08,4.472863e-08,0.000090,4.477336e-05
UH,1.030439e-05,1.030439e-05,1.030439e-05,0.061837,3.092348e-02


In [19]:
B = get_emission_mat(emission_counts, tag_counts, list(vocab), alpha)
emission_counts, tag_counts, vocab, alpha

print(f"View Matrix position at row 0, column 0: {B[0,0]:.9f}")
print(f"View Matrix position at row 3, column 1: {B[3,1]:.9f}")

View Matrix position at row 0, column 0: 0.000006032
View Matrix position at row 3, column 1: 0.000000720


In [ ]:
cidx  = ['725','adroitly','engineers', 'promoted', 'synergy']

# Get the integer ID for each word
cols = [vocab[a] for a in cidx]

rvals =['CD','NN','NNS', 'VB','RB','RP']

# For each POS tag, get the row number from the 'states' list
rows = [states.index(a) for a in rvals]

# Get the emissions for the sample of words, and the sample of POS tags
B_sub = pd.DataFrame(B[np.ix_(rows,cols)], index=rvals, columns = cidx )
print(B_sub)  

              725      adroitly     engineers      promoted       synergy
CD   8.201296e-05  2.732854e-08  2.732854e-08  2.732854e-08  2.732854e-08
NN   7.521128e-09  7.521128e-09  7.521128e-09  7.521128e-09  2.257091e-05
NNS  1.670013e-08  1.670013e-08  4.676203e-04  1.670013e-08  1.670013e-08
VB   3.779036e-08  3.779036e-08  3.779036e-08  3.779036e-08  3.779036e-08
RB   3.226454e-08  6.456135e-05  3.226454e-08  3.226454e-08  3.226454e-08
RP   3.723317e-07  3.723317e-07  3.723317e-07  3.723317e-07  3.723317e-07
